In [10]:
# We'll need numpy for some mathematical operations
import numpy as np
import sys

import librosa


#audio_path = "/Users/willian/Github/OSU-Mapper/osu-audio-feature-extract/Data/Beatmaps/355132 KOTOKO - BLAZE/fluff.mp3"
#audio_path = "/Users/willian/Github/OSU-Mapper/osu-audio-feature-extract/Data/Beatmaps/406217 Chata - enn/ehh.mp3"
audio_path = "/Applications/osu!.app/Contents/Resources/drive_c/osu!/Songs/13019 Daisuke Achiwa - BASARA/BASARA.mp3"
#audio_path = "/Applications/osu!.app/Contents/Resources/drive_c/osu!/Songs/151878 Chasers - Lost/Chasers - Lost.mp3"

y, sr = librosa.load(audio_path,sr=None)

o_env = librosa.onset.onset_strength(y, sr=sr)

dtempo = librosa.beat.tempo(onset_envelope=o_env, sr=sr,
                            aggregate=None)
# print (len(dtempo))

times = librosa.frames_to_time(np.arange(len(o_env)), sr=sr)

arr_bpm = []
pre = dtempo[0]
# print (pre)
for i in range(1, len(dtempo)):
    tmp = dtempo[i]
    if (dtempo[i] != pre):
        arr_bpm.append((60/pre,times[i]))
        pre = tmp

arr_bpm.append((60/pre, times[len(dtempo)-1]))
#print (arr_bpm)
#arr_bpm = [(spb, endtime)]
arr_bpm

[(0.24380952380952381, 0.40634920634920635),
 (0.47600907029478456, 1.3931972789115645),
 (0.46439909297052157, 204.2659410430839),
 (0.45278911564625851, 204.99736961451248),
 (0.4295691609977324, 205.27600907029478),
 (0.4179591836734694, 205.8332879818594),
 (0.3831292517006803, 206.51827664399093)]

In [7]:
from itertools import islice

def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [8]:
hop_length= 512
onset_times = librosa.onset.onset_detect(y=y, sr=sr, hop_length=hop_length, precise=True,
                                         units='time')

#onset_times = times[onset_frames]

whichBPM = 0
thisTuple = arr_bpm[whichBPM]
lastTuple = (0,0);
arr_statistics = []
thisOnset = [0]
aggragatedOnsets = []
thisOnsets = []
# print(onset_times)
for onsetTime in onset_times:
    (thisSPB, thisEndtime) = thisTuple
#     print(thisTuple)
    if onsetTime > thisEndtime:
        aggragatedOnsets.append(thisOnsets)
#         arr_statistics.append([lastTuple[1]*1000, 60/thisTuple[0], int(np.median(thisOnset)*1000)])
#         lastTuple = arr_bpm[whichBPM]
        thisOnsets = []
        whichBPM += 1
        thisTuple = arr_bpm[whichBPM]
    thisOnsets.append(onsetTime)
    #thisSPB = thisTuple[0]
#     thisOnset = (onsetTime-lastTuple[1]) % thisSPB
aggragatedOnsets.append(thisOnsets)
# arr_statistics.append([lastTuple[1]*1000, 60/thisTuple[0], int(np.median(thisOnset)*1000)])

#[startTime, BPM, offset]
# arr_statistics
#aggragatedOnsets

In [9]:
from itertools import compress
from collections import Counter
for onsets in aggragatedOnsets:
    print("beats: ",len(onsets))
    if(len(onsets)>2):
        windowsize = 1
        hop_length = 512
        timeIntervals = [(w[windowsize]-w[0])/windowsize for w in window(onsets, windowsize+1)]
        #quantiles = range(1,100,10)
        #print(list(quantiles))
        #print(60/np.percentile(timeIntervals, q=quantiles))
        
        #
        median = np.median(timeIntervals)
        print("median: ",30/median)
        bpm = 60/median
        theshold = (60/bpm)-(60/(bpm*1.10))
        bigThreshold = median + theshold
        smallThreshold = median - theshold
        #
        for i in range(0,2):
            timeIntervals = [interval/2 if interval > median*1.5 else interval
                            for interval
                            in timeIntervals ]
#         for i in range(0,3):
#             timeIntervals = [interval*2 if interval < median/1.5 else interval
#                             for interval
#                             in timeIntervals ]
        #
        timeIntervals = [interval
                        for interval
                        in timeIntervals 
                        if interval<bigThreshold and interval>smallThreshold]

        #
        timeIntervals = [30/inteval for inteval in timeIntervals]
        #print("count: ",Counter(timeIntervals))
        #print(np.percentile(timeIntervals, q=quantiles))
        print("newBPM: ", np.mean(timeIntervals))
        #print(set(framesIntervalForBPM))

beats:  3
median:  164.716135458
newBPM:  165.159174465
beats:  100
median:  169.441598361
newBPM:  170.275698105
beats:  152
median:  170.841942149
newBPM:  170.140316665
beats:  1
beats:  1
beats:  1
beats:  1
beats:  1
beats:  1
beats:  1
beats:  1
beats:  6
median:  85.4209710744
newBPM:  85.5391190288
beats:  4
median:  84.0320121951
newBPM:  84.9041235691
beats:  71
median:  86.1328125
newBPM:  85.1569699392
beats:  5
median:  112.347146739
newBPM:  nan
beats:  2
beats:  5
median:  84.8947638604
newBPM:  84.8994176894
beats:  10
median:  160.247093023
newBPM:  169.680946466
beats:  10
median:  166.708669355
newBPM:  170.202946069
beats:  1
beats:  394
median:  170.841942149
newBPM:  170.211184542
beats:  90
median:  170.841942149
newBPM:  170.334800089
beats:  23
median:  169.441598361
newBPM:  169.83974523
beats:  10
median:  95.703125
newBPM:  95.9318879577


/Users/willian/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/Users/willian/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# Useless

In [81]:
for onsets in aggragatedOnsets:
    if(len(onsets)):
        print(60/np.median([w[1]-w[0] for w in window(onsets, 2)]))

In [145]:
interval = 1
hop_length = 512
onset_frames = librosa.onset.onset_detect(onset_envelope=o_env, sr=sr, hop_length=hop_length)    
onset_times = librosa.frames_to_time(onset_frames,sr=sr, hop_length=hop_length)
framesIntervalForBPM = [(w[interval]-w[0])/interval for w in window(onset_times, interval+1)]
#print(framesIntervalForBPM)
print(set(framesIntervalForBPM))

{0.24380952380952392, 0.1160997732426301, 0.10448979591836727, 0.10448979591836771, 0.12770975056689338, 0.11609977324263099, 0.16253968253968232, 0.069659863945577882, 0.22058956916099781, 0.24380952380952348, 0.24380952380952436, 0.10448979591836682, 0.1044897959183686, 0.2089795918367372, 0.19736961451247126, 0.70820861678004121, 0.46439909297049553, 0.081269841269829612, 0.092879818594104435, 4.0402721088435385, 0.092879818594106212, 0.13931972789114866, 0.092879818594099106, 0.046439909297049553, 0.081269841269840271, 0.47600907029477924, 0.46439909297050974, 0.68498866213151643, 0.058049886621319047, 0.68498866213151999, 0.34829931972788586, 0.46439909297051685, 0.26702947845805092, 0.11609977324263809, 0.4643990929705204, 0.12770975056689426, 0.35990929705215535, 0.46439909297052395, 0.25541950113378675, 0.45278911564625446, 0.24380952380951726, 0.22058956916100669, 0.23219954648524777, 0.11609977324260967, 0.12770975056687917, 0.24380952380952081, 0.45278911564625801, 0.1277097

In [139]:
librosa.beat.beat_track(y=y, sr=sr, units='time')

(129.19921875,
 array([   2.65868481,    3.1230839 ,    3.59909297,    4.06349206,
           4.5046712 ,    4.96907029,    5.44507937,    5.90947846,
           6.36226757,    6.82666667,    7.29106576,    7.76707483,
           8.2430839 ,    8.67265306,    9.16027211,    9.61306122,
          10.05424036,   10.51863946,   10.98303855,   11.44743764,
          11.90022676,   12.36462585,   12.81741497,   13.28181406,
          13.74621315,   14.21061224,   14.67501134,   15.13941043,
          15.59219955,   16.05659864,   16.52099773,   16.98539683,
          17.43818594,   17.90258503,   18.36698413,   18.83138322,
          19.28417234,   19.74857143,   20.21297052,   20.67736961,
          21.13015873,   21.59455782,   22.05895692,   22.52335601,
          22.97614512,   23.44054422,   23.90494331,   24.3693424 ,
          24.82213152,   25.28653061,   25.75092971,   26.2153288 ,
          26.66811791,   27.13251701,   27.5969161 ,   28.06131519,
          28.51410431,   28.97850

In [155]:
arr_bpm


[(0.24380952380952381, 0.40634920634920635),
 (0.47600907029478456, 1.3931972789115645),
 (0.46439909297052157, 204.2659410430839),
 (0.45278911564625851, 204.99736961451248),
 (0.4295691609977324, 205.27600907029478),
 (0.4179591836734694, 205.8332879818594),
 (0.3831292517006803, 206.51827664399093)]